In [1]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

Now import the MNIST data set which fortunately is included in TFLearn

In [2]:
import tflearn.datasets.mnist as mnist

# Convolutional Neural Network

For proper evaluation, we now split into train and test sets and prepare the predictors for the network. As the names imply, the train set will be used for training the network and test set will be used to evaluate it's performance after training.

In [3]:
X_train, y_train, X_test, y_test = mnist.load_data(one_hot=True)
X_train = X_train.reshape([-1, 28, 28, 1])
X_test = X_test.reshape([-1, 28, 28, 1])

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


Here we construct the convolutional neural network

In [4]:
cnn = input_data(shape=[None, 28, 28, 1], name='input')
cnn = conv_2d(cnn, 32, 3, activation='relu', regularizer="L2")
cnn = max_pool_2d(cnn, 2)
cnn = local_response_normalization(cnn)

cnn = conv_2d(cnn, 64, 3, activation='relu', regularizer="L2")
cnn = max_pool_2d(cnn, 2)

cnn = local_response_normalization(cnn)
cnn = fully_connected(cnn, 128, activation='tanh')

cnn = dropout(cnn, 0.8)
cnn = fully_connected(cnn, 256, activation='tanh')
cnn = dropout(cnn, 0.8)
cnn = fully_connected(cnn, 10, activation='softmax')

cnn = regression(cnn, optimizer='sgd', learning_rate=0.01, loss='categorical_crossentropy', name='target')

Now we hand the training data over to the network for training

In [5]:
mdl = tflearn.DNN(cnn, tensorboard_verbose=0)
mdl.fit({'input': X_train}, {'target': y_train}, n_epoch=10, validation_set=({'input': X_test}, {'target': y_test}), snapshot_step=100, show_metric=True, run_id='cnn_mnist')

Training Step: 8599  | total loss: 0.15558 | time: 239.450s
| SGD | epoch: 010 | loss: 0.15558 - acc: 0.9763 -- iter: 54976/55000
Training Step: 8600  | total loss: 0.15130 | time: 249.156s
| SGD | epoch: 010 | loss: 0.15130 - acc: 0.9739 | val_loss: 0.06213 - val_acc: 0.9796 -- iter: 55000/55000
--


A quick 10 epochs gets us to 98% accuracy